In [1]:
import pandas as pd
from nltk.corpus import comparative_sentences  #keyword comparison 생성 모듈

data = pd.read_csv('reviews_preprocessing.csv', engine='python')
data

,asin,date,product,review,score,user
0,B00000J48G,"11 18, 1999",pdc-640,i have taken over 500 pics using all three foc...,4.0,A.J.Kirchoff (xgi@ametro.net)
1,B00000J48G,"11 18, 1999",pdc-640,i wish that it would take the 16memory and th...,4.0,A.J.Kirchoff (xgi@ametro.net)
2,B00000J48G,"06 10, 2002",pdc-640,only draw back is theres no optical lens.,4.0,"Andrea Rowland ""Andrea"""
3,B00000J48G,"04 26, 2000",pdc-640,i like the way my 5x7 prints and smaller turn...,4.0,"""denisey65"""
4,B00000J48G,"04 26, 2000",pdc-640,"flashing is important in any photography, and...",4.0,"""denisey65"""
5,B00000J48G,"04 26, 2000",pdc-640,you definitely cannot rush the recharge on th...,4.0,"""denisey65"""
6,B00000J48G,"04 26, 2000",pdc-640,also- the &quot;video&quot; function on the l...,4.0,"""denisey65"""
7,B00000J48G,"04 26, 2000",pdc-640,i bought an optional flashpath adapter that a...,4.0,"""denisey65"""
8,B00000J48G,"10 11, 2000",pdc-640,a flashpath floppy disk adapter makes it easy ...,5.0,"Dennis Frank ""djarchivist"""
9,B00000J48G,"02 21, 2000",pdc-640,"anyway, this camera quite good, but i thought...",4.0,d


In [2]:
#데이터 확인
len(data)

367553

In [3]:
#kw 사전 리스트 생성
### kw_list = comparative_sentences.keywords_readme()[5:]
### kw_str = '|'.join(kw_list)

#kw 사전 수작업으로 분류 후 다시 읽어오기
def call_kw():
    kw_list = pd.read_csv('kw_list.csv',engine='python')
    #칼럼 중에 'to_use'만 가져오기
    kw_list = list(kw_list['to_use'])
#     kw_str = '|'.join(kw_list)
    
    return kw_list

for kw in call_kw():
    print(kw)

beat
inferior
outstrip
Choice
choose
prefer
recommend
outperform
superior
all
favor
defeat
twice
thrice
Number one
more
like
Versus
first
outdistance
before
double
outsell
nobody
Vs
last
after
thrice
improve
only
outmatch
ahead
fraction
outdo
unmatched
peerless
differ
outwit
Compare
top
exceed
lead
win
outstrip
none
unrivaled
dominate
second
nonpareil
advantage
unlike
least
outclass
outfox
outdistance
most
up against
less
half
behind
Compete


In [4]:
len(call_kw())

61

In [5]:
#CS / SS 라벨링
kw_list=call_kw()
def tokenizedNtagging(text):
    #kw로 분류
    for kw in kw_list:
        if kw.lower() in text:
            return 'CS'
        
    #pos로 분류    
    from nltk import word_tokenize, pos_tag
    # nltk로 토크나이즈, 포스태깅
    tokens = word_tokenize(text)
    pos_tokens = pos_tag(tokens)

    count = 0
    for tok, pos in pos_tokens:
        if pos in ['JJR', 'JJS', 'RBS', "RBR"]:
            count+=1

    if count != 0:
        return 'CS'
    else:
        return 'SS'            

In [6]:
#모델 사전 호출 함수
def call_model():
    df=pd.read_csv('product_list.csv', engine='python')
    model_ls=list(df['product'])
    model_reg='( '+' | '.join(model_ls)+' )'
    return model_reg

In [ ]:
#자기 자신을 언급한 제품 삭제
def targetmodel_del(df):
    import numpy as np
    def same_del(p,t):
        tmp = t.split(',')
        if p in tmp:
            print("{} : {} 발견, 제거".format(tmp,p))
            tmp.remove(p)

        if len(tmp) == 0:
            return np.nan

        return ','.join(tmp)
    df['target']=df.apply(lambda x:same_del(x['product'],x['target']),axis=1)
    
    df=df.dropna()
    return df

In [ ]:
#csss 라벨링, 타겟 제품 검색 모델링
def make_sscs(data):
    product_ls=list(set(list(data['product'])))
    
    data['CS_SS']=data['review']
    data['target']=data['review']
    data['CS_SS']=data['CS_SS'].apply(tokenizedNtagging)
    print('CS_SS 라벨링 끝')
    data_cs=data[data['CS_SS']=='CS']
    data_ss=data[data['CS_SS']=='SS']
    data_ss['target']=data_ss['target'].apply(lambda x:'NaN')
    data_cs['target']=data_cs['target'].apply(targetmodel_read)
    print('모델명 탐색 끝')
    data_cs=data_cs.dropna()
    data_cs=targetmodel_del(data_cs)
    print('자기 자신 호출 CS 제거')
    new_data=pd.concat([data_cs,data_ss])
    return new_data

In [ ]:
df_t = make_sscs(data)
df_t.head()

In [ ]:
len(df_t[df_t.CS_SS=='CS'])

In [ ]:
len(df_t[df_t.CS_SS=='SS'])

In [ ]:
df_t.to_csv('labelingSSCS.csv',index=False, encoding='utf-8')